In [50]:

# === Imports ===
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sns.set_style("whitegrid")


In [51]:
df = pd.read_csv("fremont_df.csv")
df.columns = ['Date', 'Total', 'West', 'East']
df.isnull().sum()
df = df.dropna()  

In [52]:
df['Date'] = pd.to_datetime(df['Date'])

C:\Users\conor\AppData\Local\Temp\ipykernel_1224\2394721818.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [53]:
df['Hour'] = df['Date'].dt.hour
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Weekday'] = df['Date'].dt.day_name()
df['Weekend'] = df['Weekday'].isin(['Saturday', 'Sunday'])

In [54]:
# Group by Month & Year
monthly = df.groupby([df['Year'], df['Month']])['Total'].sum().reset_index()

In [55]:
# Create df column for plotting
monthly['Date'] = pd.to_datetime(monthly[['Year', 'Month']].assign(DAY=1))

In [56]:
# Group by Weekday
weekday_totals = df.groupby('Weekday')['Total'].sum().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

In [57]:
# Group by hour, average counts
hourly = df.groupby('Hour')[['East', 'West']].mean()

In [58]:
# Create a 'WeekType' column
df['WeekType'] = df['Weekend'].map({True: 'Weekend', False: 'Weekday'})

In [59]:
# Group by WeekType and Hour, then average total counts
hourly_trend = df.groupby(['WeekType', 'Hour'])[['East', 'West']].mean()

In [60]:
# Sum East + West to get total sidewalk use
hourly_trend['Total'] = hourly_trend['East'] + hourly_trend['West']

In [61]:
# Reset index for plotting
hourly_trend = hourly_trend.reset_index()

In [62]:
yearly_counts = df.groupby('Year')['Total'].sum().reset_index()

In [63]:
# === Filters ===
year_slider = widgets.IntRangeSlider(
    value=[df['Year'].min(), df['Year'].max()],
    min=df['Year'].min(),
    max=df['Year'].max(),
    step=1,
    description='Year Range:',
    continuous_update=False,
    layout=widgets.Layout(width='95%')
)

week_filter = widgets.ToggleButtons(
    options=['All', 'Weekday', 'Weekend'],
    value='All',
    description='Traffic Type:'
)

side_filter = widgets.Dropdown(
    options=['Total', 'East Sidewalk', 'West Sidewalk'],
    value='Total',
    description='Direction:'
)


In [64]:
# Outputs
out1, out2, out3, out4, out5, out6 = [widgets.Output() for _ in range(6)]

# Filter Helper
def filter_df():
    dff = df.copy()
    y0, y1 = year_slider.value
    dff = dff[(dff['Year'] >= y0) & (dff['Year'] <= y1)]
    if week_filter.value == 'Weekday':
        dff = dff[~dff['Weekend']]
    elif week_filter.value == 'Weekend':
        dff = dff[dff['Weekend']]
    return dff


In [65]:
def plot_monthly(out):
    with out:
        clear_output(wait=True)
        dff = filter_df()
        monthly_data = dff.resample('ME', on='Date').sum()
        plt.figure(figsize=(6, 4))
        plt.plot(monthly_data.index, monthly_data['Total'], marker='o')
        plt.title("Monthly Bicycle Traffic Over Time")
        plt.xlabel("Date")
        plt.ylabel("Total Count")
        plt.tight_layout()
        plt.show()

def plot_dayofweek(out):
    with out:
        clear_output(wait=True)
        dff = filter_df()
        order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
        agg = dff.groupby('Weekday')['Total'].sum().reindex(order)
        plt.figure(figsize=(6, 4))
        sns.barplot(x=agg.index, y=agg.values, color="skyblue")
        plt.title("Total Traffic by Day of Week")
        plt.ylabel("Total Count")
        plt.tight_layout()
        plt.show()

def plot_hourly_direction(out):
    with out:
        clear_output(wait=True)
        dff = filter_df()
        hourly = dff.groupby('Hour')[['East','West']].mean()
        plt.figure(figsize=(6, 4))
        plt.plot(hourly.index, hourly['East'], marker='o', label='East')
        plt.plot(hourly.index, hourly['West'], marker='o', label='West')
        plt.title("Average Hourly Bicycle Traffic (East vs West)")
        plt.xlabel("Hour")
        plt.ylabel("Average Count")
        plt.legend()
        plt.tight_layout()
        plt.show()


In [66]:
def plot_hourly_weektype(out):
    with out:
        clear_output(wait=True)
        dff = filter_df()
        dff['WeekType'] = dff['Weekend'].map({True: 'Weekend', False: 'Weekday'})
        pivot = dff.groupby(['Hour', 'WeekType'])['Total'].mean().unstack()
        pivot.plot(figsize=(6, 4), marker='o')
        plt.title("Avg Hourly Bicycle Traffic: Weekday vs Weekend")
        plt.xlabel("Hour")
        plt.ylabel("Avg Count")
        plt.tight_layout()
        plt.show()

def plot_yearly(out):
    with out:
        clear_output(wait=True)
        dff = filter_df()
        yearly_totals = dff.groupby('Year')['Total'].sum()
        plt.figure(figsize=(6, 4))
        sns.barplot(x=yearly_totals.index, y=yearly_totals.values, palette="Blues")
        plt.title("Total Bicycle Counts by Year")
        plt.xlabel("Year")
        plt.ylabel("Total Count")
        plt.tight_layout()
        plt.show()

def plot_heatmap(out):
    with out:
        clear_output(wait=True)
        dff = filter_df()
        pivot = dff.pivot_table(index='Weekday', columns='Hour', values='Total', aggfunc='mean')
        order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
        pivot = pivot.reindex(order)
        plt.figure(figsize=(6, 4))
        sns.heatmap(pivot, cmap="YlGnBu")
        plt.title("Avg Bicycle Count by Hour and Weekday")
        plt.xlabel("Hour")
        plt.ylabel("Weekday")
        plt.tight_layout()
        plt.show()


In [67]:
def update_all(*args):
    plot_monthly(out1)
    plot_dayofweek(out2)
    plot_hourly_direction(out3)
    plot_hourly_weektype(out4)
    plot_yearly(out5)
    plot_heatmap(out6)

for w in [year_slider, week_filter, side_filter]:
    w.observe(update_all, 'value')

top_controls = widgets.VBox([year_slider, widgets.HBox([week_filter, side_filter])])
grid = widgets.VBox([
    widgets.HBox([out1, out2]),
    widgets.HBox([out3, out4]),
    widgets.HBox([out5, out6]),
])

title = widgets.HTML("<h2 style='text-align:center;'>Fremont Bridge Bicycle Dashboard</h2>")
dashboard = widgets.VBox([title, top_controls, grid])

update_all()
display(dashboard)
